# 📧 Phishing Email Analyzer
Analyze suspicious email headers and links to detect phishing attempts.

This tool checks:
- Mismatched `From` and `Reply-To` fields
- Suspicious URLs
- Shortened links
- IP geolocation of sender

In [ ]:
# 📦 Install Required Libraries
!pip install tldextract ipwhois requests

In [ ]:
# 🛡️ Email Header & Link Analyzer
import re
import tldextract
import requests
from ipwhois import IPWhois

def analyze_email(header, body_text):
    print("\n🔍 Analyzing Email Header...")
    
    from_match = re.search(r'From: .*<(.*)>', header)
    reply_to_match = re.search(r'Reply-To: .*<(.*)>', header)
    ip_match = re.search(r'Received: from .*\[(\d+\.\d+\.\d+\.\d+)\]', header)

    if from_match and reply_to_match:
        from_email = from_match.group(1)
        reply_to = reply_to_match.group(1)
        print(f"✉️ From: {from_email}")
        print(f"📩 Reply-To: {reply_to}")
        if from_email != reply_to:
            print("⚠️ Mismatch between 'From' and 'Reply-To' — possible spoofing!")
    else:
        print("❌ Could not extract 'From' or 'Reply-To' fields.")

    if ip_match:
        ip = ip_match.group(1)
        print(f"🌐 Sender IP: {ip}")
        try:
            obj = IPWhois(ip)
            res = obj.lookup_rdap()
            print("🏢 Org:", res.get('network', {}).get('name'))
            print("📍 Country:", res.get('network', {}).get('country'))
        except:
            print("⚠️ IP lookup failed")
    else:
        print("⚠️ Could not extract sender IP.")

    print("\n🔗 Checking for Suspicious Links...")
    urls = re.findall(r'https?://\S+', body_text)
    for url in urls:
        domain = tldextract.extract(url).registered_domain
        if "bit.ly" in url or "tinyurl" in url:
            print(f"⚠️ Shortened URL detected: {url}")
        elif domain in ["paypal-login-verification.com", "goog1e-verify.com"]:
            print(f"🚨 Known suspicious domain: {url}")
        else:
            print(f"🔗 URL found: {url}")

In [ ]:
# 🧪 Example Usage
sample_header = '''From: PayPal <support@paypal.com>
Reply-To: help@paypal-login-verification.com
Received: from [45.76.123.1] by mail.fake.net;'''

sample_body = '''Please confirm your account: https://paypal-login-verification.com/confirm
Or visit our secure page: https://bit.ly/fakepaypal'''

analyze_email(sample_header, sample_body)

⚠️ You can replace the sample header and body with your own test data.